<a href="https://colab.research.google.com/github/psasanka1729/gpu_programs/blob/main/spectrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cupy as cp
import cupyx.scipy.sparse as cupy_sparse
from cupyx.scipy.sparse.linalg import eigsh # Import the sparse eigensolver
import numpy as np
import os
import time
from itertools import combinations, product
from scipy.sparse import load_npz, save_npz, csr_matrix

def construct_H_mu_sigma_gpu(mu, sigma, H_1_0, H_0_1):
     """Constructs the Hamiltonian on the GPU."""
     return mu * H_1_0 + sigma * H_0_1

if __name__ == "__main__":
    try:
        device_id = cp.cuda.runtime.getDevice()
        print(f"✅ GPU Device {device_id} detected. Running on GPU.")
    except cp.cuda.runtime.CUDARuntimeError as e:
        print(f"GPU not detected or CuPy installation issue. Error: {e}")
        exit()

    # --- 1. Set Parameters ---
    L = 15
    k = 7
    # Total number of eigenvalues to find around the center of the spectrum.
    # MUST BE AN EVEN NUMBER.
    TOTAL_EIGENVALUES_TO_FIND = 500

    OUTPUT_DIR = "spectrum"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Parameters: L={L}, k={k}. Output will be saved in '{OUTPUT_DIR}/'")
    print(f"Finding the {TOTAL_EIGENVALUES_TO_FIND} eigenvalues closest to zero using spectral symmetry.")

    # --- 2. Load Hamiltonian Components ---
    print("Loading Hamiltonian components...")
    try:
        # Create dummy files for demonstration if they don't exist
        if not os.path.exists(f'H_1_0_L_{L}_k_{k}.npz'):
            print("Creating dummy Hamiltonian files...")
            dummy_H = cupy_sparse.random(2**L, 2**L, density=0.0001, format='csr', dtype=cp.complex128)
            dummy_H = dummy_H + dummy_H.T.conj() # Make it Hermitian
            save_npz(f'H_1_0_L_{L}_k_{k}.npz', dummy_H.get())
            save_npz(f'H_0_1_random_seed_1729_L_{L}_k_{k}.npz', dummy_H.get())

        H_1_0_cpu = load_npz(f'H_1_0_L_{L}_k_{k}.npz')
        H_0_1_cpu = load_npz(f'H_0_1_random_seed_1729_L_{L}_k_{k}.npz')

        # Move to GPU
        H_1_0_gpu = cupy_sparse.csr_matrix(H_1_0_cpu, dtype=cp.complex128)
        H_0_1_gpu = cupy_sparse.csr_matrix(H_0_1_cpu, dtype=cp.complex128)
        print("Hamiltonian components loaded to GPU.")
    except FileNotFoundError:
        print("ERROR: Hamiltonian component files not found. Please generate them first.")
        exit()

    # --- 3. Main Loop ---
    sigma_lst = np.linspace(0.0, 2.0, 100)

    for sigma_val in sigma_lst:
        sigma = np.around(sigma_val, 5)
        print(f"\n--- Processing for sigma = {sigma} ---")

        H_gpu = construct_H_mu_sigma_gpu(1.0, sigma, H_1_0_gpu, H_0_1_gpu)

        # --- MEMORY-EFFICIENT DIAGONALIZATION USING SYMMETRY ---
        num_negative_to_find = TOTAL_EIGENVALUES_TO_FIND // 2
        print(f"Finding {num_negative_to_find} most negative eigenvalues...")
        start_diag_time = time.time()

        # Use which='SA' to get the eigenvalues with the Smallest Algebraic value (most negative).
        negative_eigenvalues_gpu = eigsh(
            H_gpu, k=num_negative_to_find, which='SA', return_eigenvectors=False
        )

        cp.cuda.Stream.null.synchronize()
        end_diag_time = time.time()
        print(f"Diagonalization finished in {end_diag_time - start_diag_time:.2f} seconds.")

        # --- 4. Construct Full Spectrum and Save ---
        # Use symmetry to find the positive counterparts.
        positive_eigenvalues_gpu = -negative_eigenvalues_gpu

        # Combine the negative and positive parts.
        all_eigenvalues_gpu = cp.concatenate((negative_eigenvalues_gpu, positive_eigenvalues_gpu))
        all_eigenvalues_gpu = cp.sort(all_eigenvalues_gpu) # Sort for clean output

        # Move to CPU for saving
        eigenvalues_cpu = cp.asnumpy(all_eigenvalues_gpu)
        # eigenvectors_cpu = cp.asnumpy(eigenvectors)

        eigvals_path = os.path.join(OUTPUT_DIR, f'eigenvalues_L_{L}_k_{k}_mu_1_sigma_{sigma}.npz')
        save_npz(eigvals_path, csr_matrix(eigenvalues_cpu))
        # eigvecs_path = os.path.join(OUTPUT_DIR, f'eigenvectors_L_{L}_k_{k}_mu_1_sigma_{sigma}.npz')
        # save_npz(eigvecs_path, csr_matrix(eigenvectors_cpu))
        print(f"Results for sigma={sigma} saved successfully.")

    print("\nAll calculations are complete.")

✅ GPU Device 0 detected. Running on GPU.
Parameters: L=15, k=7. Output will be saved in 'spectrum/'
Finding the 500 eigenvalues closest to zero using spectral symmetry.
Loading Hamiltonian components...
Hamiltonian components loaded to GPU.

--- Processing for sigma = 0.0 ---
Finding 250 most negative eigenvalues...
Diagonalization finished in 381.39 seconds.
Results for sigma=0.0 saved successfully.

--- Processing for sigma = 0.0202 ---
Finding 250 most negative eigenvalues...
Diagonalization finished in 68.99 seconds.
Results for sigma=0.0202 saved successfully.

--- Processing for sigma = 0.0404 ---
Finding 250 most negative eigenvalues...
Diagonalization finished in 69.86 seconds.
Results for sigma=0.0404 saved successfully.

--- Processing for sigma = 0.06061 ---
Finding 250 most negative eigenvalues...
Diagonalization finished in 69.00 seconds.
Results for sigma=0.06061 saved successfully.

--- Processing for sigma = 0.08081 ---
Finding 250 most negative eigenvalues...
Diagonali